# Lab 1: Introduction and Docker-Based Deployment

## Overview

In this lab, you will:
- Set up the Dynamo environment
- Deploy Dynamo using Docker-based aggregated deployment
- Configure a backend engine (to be determined during the lab)
- Benchmark the deployment using AI-Perf

## Duration: ~90 minutes

---


## Section 1: Environment Setup

### Objectives
- Install prerequisites (Docker, Python, uv)
- Verify system requirements (GPU availability)
- Install Dynamo dependencies

### Tasks
- [ ] Check Python version
- [ ] Install Docker and Docker Compose
- [ ] Install uv package manager
- [ ] Verify GPU access
- [ ] Install Python development headers


## Section 2: Docker-Based Aggregated Deployment

### Objectives
- Understand aggregated serving architecture
- Deploy etcd and NATS using Docker Compose
- Deploy Dynamo frontend and router
- Deploy inference workers with selected backend

### Architecture
```
Client → Frontend → Router → Worker(s) with Backend Engine
                        ↓
                   etcd + NATS
```

### Tasks
- [ ] Review Docker Compose configuration
- [ ] Start etcd and NATS services
- [ ] Deploy Dynamo frontend
- [ ] Select and deploy backend engine (vLLM/SGLang/TensorRT-LLM)
- [ ] Verify deployment health


## Section 3: Backend Engine Selection

### Available Backends
- **vLLM**: High-throughput serving with PagedAttention
- **SGLang**: Optimized for complex prompting and structured generation
- **TensorRT-LLM**: Maximum performance on NVIDIA GPUs

### Tasks
- [ ] Review backend engine options
- [ ] Select appropriate backend for use case
- [ ] Configure backend-specific parameters
- [ ] Deploy selected backend
- [ ] Test basic inference


## Section 4: Testing and Validation

### Objectives
- Send test requests to the deployment
- Verify OpenAI API compatibility
- Test streaming and non-streaming responses

### Tasks
- [ ] Send sample chat completion request
- [ ] Test streaming responses
- [ ] Verify response format
- [ ] Test different parameters (temperature, max_tokens, etc.)


## Section 5: Benchmarking with AI-Perf

### Objectives
- Install and configure AI-Perf benchmarking tool
- Run performance benchmarks
- Analyze throughput, latency, and token metrics
- Compare performance across different configurations

### Metrics to Measure
- Throughput (requests/second, tokens/second)
- Latency (TTFT, TPOT, end-to-end)
- GPU utilization
- KV cache efficiency

### Tasks
- [ ] Install AI-Perf
- [ ] Configure benchmark parameters
- [ ] Run baseline benchmark
- [ ] Analyze results
- [ ] Experiment with different load patterns


## Section 6: Exercises and Exploration

### Exercise 1: Parameter Tuning
- Adjust backend engine parameters
- Measure impact on performance

### Exercise 2: Load Testing
- Test with increasing concurrent requests
- Identify bottlenecks

### Exercise 3: Comparison
- Try different backend engines
- Compare performance characteristics


## Summary

### What You Learned
- ✅ How to set up Dynamo environment
- ✅ Docker-based aggregated deployment architecture
- ✅ Backend engine selection and configuration
- ✅ Performance benchmarking with AI-Perf

### Key Takeaways
- Aggregated serving is simpler to deploy and manage
- Different backends have different performance characteristics
- AI-Perf provides comprehensive performance insights

### Next Steps
In **Lab 2**, you'll deploy Dynamo on Kubernetes with both aggregated and disaggregated serving topologies.
